In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from math import isnan

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
df = pd.read_csv('FullData.csv')
df = df.apply(lambda x: x.astype(str).str.lower())
df = df.drop('Country', axis=1)
df = df.drop('I agree with the terms mentioned above:', axis=1)
df.head()

,Age,Sex,Marital Status,Education,#1,# 2,# 3,# 4,# 5,# 6,...,# 8,# 9,# 10,# 11,# 12,O,C,E,A,N
0,48,female,divorced,high school degree or equivalent,gemstone,waterfall,clouds,bacon,water,wave,...,mountains and rivers,volcano,tide,space,explosion,2,5,1,4,3
1,23,female,single never married,bachelor's degree (eg ba bs),planet,galaxy,sea,colored water,rocks,feather,...,land,planet,ursulla,sand and sea,?,3,4,3,4,3
2,26,male,single never married,high school degree or equivalent,galaxy,weird,lock screen photo,ameba,glass,feather,...,earth,australia,3d,sand and sea,?,3,4,2,4,3
3,21,female,single never married,high school degree or equivalent,a distant photograph of israel,waves in the sea,waves in the sea,cell development,sea,sea,...,earth,?,?,desert,ink that spreads in water,4,4,3,5,2
4,24,male,single never married,high school degree or equivalent,waves cell supernova,cellular structures,wavy,cells (current lesson),beach and sea,dawn,...,earth from above,a city,something dark in his nature,dunes,ghost pokémon,5,3,4,5,2


In [4]:
df['Age'].min()
df['Age'].max()

'18'

'66'

### categories:

#### sex: '0' - female, '1' - male
#### marital Status: {'0': divorced;   '1': married or domestic partnership;   '2': separated;   '3': single}
#### education: {'0': bachelor's degree;   '1': doctorate;   '2': high school degree;   '3': less than a high school diploma;   '4': master's degree;   '5': other}
#### age - {'0': 18-29;   '1': 30-39;   '2': 40-49;   '3': 50-59;   '4': 60+}

In [5]:
df['Sex'] = df['Sex'].astype('category').cat.codes
df['Marital Status'] = df['Marital Status'].astype('category').cat.codes
df['Education'] = df['Education'].astype('category').cat.codes
for index in df.index:
    df.loc[index, 'Age'] = int(df.loc[index, 'Age'])
    if df.loc[index, 'Age'] < 30:
        df.loc[index, 'Age'] = 0
    elif df.loc[index, 'Age'] >= 30 and df.loc[index, 'Age'] < 40:
        df.loc[index, 'Age'] = 1
    elif df.loc[index, 'Age'] >= 40 and df.loc[index, 'Age'] < 50:
        df.loc[index, 'Age'] = 2
    elif df.loc[index, 'Age'] >= 50 and df.loc[index, 'Age'] < 60:
        df.loc[index, 'Age'] = 3
    else:
        df.loc[index, 'Age'] = 4

df.head()

,Age,Sex,Marital Status,Education,#1,# 2,# 3,# 4,# 5,# 6,...,# 8,# 9,# 10,# 11,# 12,O,C,E,A,N
0,2,0,0,2,gemstone,waterfall,clouds,bacon,water,wave,...,mountains and rivers,volcano,tide,space,explosion,2,5,1,4,3
1,0,0,3,0,planet,galaxy,sea,colored water,rocks,feather,...,land,planet,ursulla,sand and sea,?,3,4,3,4,3
2,0,1,3,2,galaxy,weird,lock screen photo,ameba,glass,feather,...,earth,australia,3d,sand and sea,?,3,4,2,4,3
3,0,0,3,2,a distant photograph of israel,waves in the sea,waves in the sea,cell development,sea,sea,...,earth,?,?,desert,ink that spreads in water,4,4,3,5,2
4,0,1,3,2,waves cell supernova,cellular structures,wavy,cells (current lesson),beach and sea,dawn,...,earth from above,a city,something dark in his nature,dunes,ghost pokémon,5,3,4,5,2


In [6]:
pics_objects_df = df.loc[:, '#1':'# 12']
pics_objects_df.head()

,#1,# 2,# 3,# 4,# 5,# 6,# 7,# 8,# 9,# 10,# 11,# 12
0,gemstone,waterfall,clouds,bacon,water,wave,rain,mountains and rivers,volcano,tide,space,explosion
1,planet,galaxy,sea,colored water,rocks,feather,rain,land,planet,ursulla,sand and sea,?
2,galaxy,weird,lock screen photo,ameba,glass,feather,dry paint,earth,australia,3d,sand and sea,?
3,a distant photograph of israel,waves in the sea,waves in the sea,cell development,sea,sea,raindrops on the window,earth,?,?,desert,ink that spreads in water
4,waves cell supernova,cellular structures,wavy,cells (current lesson),beach and sea,dawn,flowing,earth from above,a city,something dark in his nature,dunes,ghost pokémon


In [7]:
categories_list = pd.read_csv('category list.csv')

In [8]:
feelings = pd.read_csv('feelings.csv')
categories = pd.read_csv('categories.csv')
other = pd.read_csv('other.csv')

feelings.head()
categories.head()
other.head()

,1,0
0,creative,Slight discomfort
1,amazing,Confused
2,joy,Nausea
3,Curious,emptiness
4,nice,confusion


,1,2,3,4,5,6,7,8,9,10,...,36,37,38,39,40,41,42,43,44,45
0,lava exploded,Gemstone,planet,A distant photograph of Israel,Mountain on a beach,seashell,Dead Sea in color,Sea and shoreline,Waves,Water,...,Feather,slightly dark,arrow,trail of missile,Glacier reflected,nuclear blast,forest,icecream,Layers,Rainbow
1,warmth,Colorful gem,Galaxy,Satellite photography,Mountain ranges,oyster,Sea of ​​colors,island,sea,river,...,bird feather,light caresses out of the darkness,Arrow tail,boat trail,Very nice reflection,blast,woody nature,psychotic milkshake,Layers of something,Part of the rainbow
2,Magma,energetic rock,Supernova,travel around the world,Psychedelic everest,Algae at sea,A colorful ocean,Beach,ocean,Flow,...,Peacock feathers,the light at the end of the tunnel,Arrow flies fast,trail,snowy mountain reflection,eruption,wood,banana strawberry juice,earth levels,Rainbow forming
3,Lava,Halved gemstone,Outer space,Up,Snowy mountain,NaN,Colorful sea,Sea and beach,Sea at sunset,hot springs,...,Zoom in on a feather,darkness,sharp,NaN,Mount with reflection,NaN,NaN,sweet,Atmospheric levels,NaN
4,Lava melting into sea,stone Agate,Intergalactic,earth,Mountain very beautiful,NaN,lake of watercolors,Coastline,A flood spilling into the sea,Running water,...,Pleasant feather,secret cave,edged,NaN,mountain with reflection,NaN,NaN,Candy,NaN,NaN


,-1
0,knowing what I want
1,awe
2,Slow
3,empowered dwarfed
4,new way


In [9]:
cat_dct = categories.to_dict()

for k in cat_dct:
    inter_dct = cat_dct[k]
    cat_lst = [inter_dct[i].lower() for i in inter_dct if str(inter_dct[i]) != 'nan']
    cat_dct[k] = cat_lst

cat_dct

{'1': ['lava exploded',
  'warmth',
  'magma',
  'lava',
  'lava melting into sea',
  'warm',
  'voclano',
  'heat',
  'volcano',
  'volcanic eruption'],
 '2': ['gemstone',
  'colorful gem',
  'energetic rock',
  'halved gemstone',
  'stone agate',
  'opal',
  'stone',
  'geode',
  'layers of stone',
  'stone',
  'stones',
  'crystals'],
 '3': ['planet',
  'galaxy',
  'supernova',
  'outer space',
  'intergalactic',
  'atmosphere',
  'universe',
  'calm space',
  'nebula',
  'colors from outer space',
  'smeared galaxy',
  'deep void',
  'stars',
  'night sky',
  'lost in space',
  'space',
  'colorful planet',
  'black hole',
  'meteorite  fireball',
  'moon'],
 '4': ['a distant photograph of israel',
  'satellite photography',
  'travel around the world',
  'up',
  'earth',
  'flow to unknown location',
  'photo from space',
  'interesting satellite image',
  'satellite images',
  'lock screen photo',
  'sky from space',
  'veiw from space',
  'down to earth',
  'earth from above',
 

In [10]:
feels_dct = feelings.to_dict()

for k in feels_dct:
    inter_dct = feels_dct[k]
    feels_lst = [inter_dct[i].lower() for i in inter_dct if str(inter_dct[i]) != 'nan']
    feels_dct[k] = feels_lst

feels_dct

{'1': ['creative',
  'amazing',
  'joy',
  'curious',
  'nice',
  'creativity',
  'attractive',
  'interest',
  'attraction',
  'good',
  'impression',
  'passion',
  'hope',
  'calm',
  'peace',
  'calmness',
  'neutral',
  'happyness',
  'happy',
  'optimism',
  'positive',
  'exciting',
  'energetic',
  'imagination',
  'passionate',
  'interesting',
  'interested',
  'curiosity',
  'creation',
  'very emotionally',
  'imaginary calm',
  'peaceful',
  'relaxing',
  'relaxation',
  'warmth',
  'a storm of emotions',
  'sweeps me away',
  'astonishment',
  'bursting energy',
  'supernatural mind',
  'touching infinity',
  'aspiration',
  'born new life',
  'pretty',
  'quite',
  'relaxed',
  'gentle dance',
  'deep breath',
  'fun',
  'mistiriously happy',
  'optimistic',
  'pleasure',
  'standstill',
  'overcoming',
  'freedom',
  'soothing',
  'perfection',
  'mystery',
  'peacful',
  'thoughtful',
  'excited',
  'special',
  'cheerful',
  'control',
  'soft',
  'romantic',
  'happi

In [11]:
other_dct = other.to_dict()

for k in other_dct:
    inter_dct = other_dct[k]
    other_lst = [inter_dct[i].lower() for i in inter_dct if str(inter_dct[i]) != 'nan']
    other_dct[k] = other_lst

other_dct

{'-1': ['knowing what i want',
  'awe',
  'slow',
  'empowered dwarfed',
  'new way',
  'flowing traffic',
  'movement',
  'chase',
  'time smeared',
  'untouched',
  'sex',
  'shrimp on acid',
  'flouid france  ',
  'courage',
  'intentional spread',
  'race',
  'expansion',
  'direction',
  'there is no way back',
  'air conditioning',
  'friction',
  'bacon',
  'steps',
  'havdalah',
  'open',
  'piercing',
  'hurrying',
  'hypnotist',
  'the differentiator',
  'future',
  'renaissance',
  'heavenly',
  'distance',
  'terrace road',
  'fiction movie',
  'confident',
  'high way',
  'leaving farewell',
  'accurate',
  'vision',
  'breaking a routine',
  'sleepy',
  'survival',
  'champagne',
  'part of america',
  'communication',
  'inevitable anthropy',
  'a photo film was burned',
  'ink distribution',
  'vertical augmentedreality',
  'nylon bag',
  'lots',
  'search',
  'brotherhood',
  'trimmed in flavors',
  'museum',
  'dr. horse',
  'not earthly',
  'circle of colors',
  'spi

In [12]:
for col in pics_objects_df.columns:
    for indx in pics_objects_df.index:
        words = pics_objects_df.loc[indx, col].replace('-', ' ').replace('/', ' ').replace('.', ' ').replace('  ', ' ').split(' ')
        words = list(words)
        for word in words:
            for key in cat_dct:
                for value in cat_dct[key]:
                    if word in value:
                        pics_objects_df.loc[indx, col] = key
            for key in feels_dct:
                for value in feels_dct[key]:
                    if word in value:
                        pics_objects_df.loc[indx, col] = 0
            for key in other_dct:
                for value in other_dct[key]:
                    if word in value:
                        pics_objects_df.loc[indx, col] = 0
                
pics_objects_df.head(10)

,#1,# 2,# 3,# 4,# 5,# 6,# 7,# 8,# 9,# 10,# 11,# 12
0,2,40,32,0,0,9,0,7,1,9,11,0
1,3,3,0,0,5,36,0,37,3,18,0,0
2,3,0,0,17,33,36,0,0,24,0,0,0
3,4,0,0,17,0,0,15,0,0,0,25,0
4,3,17,9,17,0,32,0,32,0,42,25,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,35,0,36,17,40,0,15,32,4,0,4,0
7,21,0,0,20,40,40,0,0,3,44,25,10
8,0,0,11,0,0,36,0,0,0,0,0,0
9,0,0,0,44,30,18,0,4,3,27,0,0


In [13]:
pics_objects_df.to_csv('after categorize.csv')

In [14]:
categories_list

,category,num
0,"lava, hot",1
1,"layers (gems, candy)",2
2,space,3
3,earth from above,4
4,mountains,5
5,Elements from sea,6
6,colored water,7
7,coast,8
8,sea,9
9,water,10


### modest categories:

#### pic 1 - space
#### pic 2 - reflection, space, sea, contrasts, window with drops
#### pic 3 - sky and horizon	
#### pic 4 - contrasts
#### pic 5 - ice
#### pic 6 - feather
#### pic 7 - window with drops	
#### pic 8 - map
#### pic 9 - storms
#### pic 10 - creatures
#### pic 11 - desert
#### pic 12 - sweets

In [15]:
pics_objects_df['#1'].value_counts().head(5)   # 3 / 11

0     74
3     10
11    10
4      5
37     4
Name: #1, dtype: int64

In [16]:
pics_objects_df['# 2'].value_counts().head(7)   # 40 / 3 / 9 / 15 / 16

0     83
40     5
16     5
9      5
15     5
3      5
11     4
Name: # 2, dtype: int64

In [17]:
pics_objects_df['# 3'].value_counts().head(5)   # 32 / 11 / 9

0     71
32    13
11    11
9     10
37     5
Name: # 3, dtype: int64

In [18]:
pics_objects_df['# 4'].value_counts().head(5)   # 16 / 17

0     70
16    10
17     9
22     5
44     5
Name: # 4, dtype: int64

In [19]:
pics_objects_df['# 5'].value_counts().head(5)   # 31 / 17

0     66
31    13
17    12
37     7
40     6
Name: # 5, dtype: int64

In [20]:
pics_objects_df['# 6'].value_counts().head(5)   # 36 / 9 / 40

0     56
36    26
9     12
40    10
32     7
Name: # 6, dtype: int64

In [21]:
pics_objects_df['# 7'].value_counts().head(5)   # 15

0     84
15    26
14     6
31     4
19     4
Name: # 7, dtype: int64

In [22]:
pics_objects_df['# 8'].value_counts().head(5)   # 21 / 4 / 8

0     63
21    16
4     13
8     10
24     7
Name: # 8, dtype: int64

In [23]:
pics_objects_df['# 9'].value_counts().head(6)   # 11 / 3 / 1 / 16

0     54
11    16
3     12
16     9
1      9
8      7
Name: # 9, dtype: int64

In [24]:
pics_objects_df['# 10'].value_counts().head(5)   # 18 / 11

0     80
11     8
18     7
4      6
27     4
Name: # 10, dtype: int64

In [25]:
pics_objects_df['# 11'].value_counts().head(5)   # 25

0     70
25    26
32     7
4      4
21     4
Name: # 11, dtype: int64

In [26]:
pics_objects_df['# 12'].value_counts().head(5)   # 43

0     81
43     9
23     6
44     5
36     4
Name: # 12, dtype: int64

In [27]:
cat_df = df.copy()
cat_df['#1'] = pics_objects_df['#1']
cat_df['# 2'] = pics_objects_df['# 2']
cat_df['# 3'] = pics_objects_df['# 3']
cat_df['# 4'] = pics_objects_df['# 4']
cat_df['# 5'] = pics_objects_df['# 5']
cat_df['# 6'] = pics_objects_df['# 6']
cat_df['# 7'] = pics_objects_df['# 7']
cat_df['# 8'] = pics_objects_df['# 8']
cat_df['# 9'] = pics_objects_df['# 9']
cat_df['# 10'] = pics_objects_df['# 10']
cat_df['# 11'] = pics_objects_df['# 11']
cat_df['# 12'] = pics_objects_df['# 12']

cat_df.head()

,Age,Sex,Marital Status,Education,#1,# 2,# 3,# 4,# 5,# 6,...,# 8,# 9,# 10,# 11,# 12,O,C,E,A,N
0,2,0,0,2,2,40,32,0,0,9,...,7,1,9,11,0,2,5,1,4,3
1,0,0,3,0,3,3,0,0,5,36,...,37,3,18,0,0,3,4,3,4,3
2,0,1,3,2,3,0,0,17,33,36,...,0,24,0,0,0,3,4,2,4,3
3,0,0,3,2,4,0,0,17,0,0,...,0,0,0,25,0,4,4,3,5,2
4,0,1,3,2,3,17,9,17,0,32,...,32,0,42,25,0,5,3,4,5,2


In [28]:
cat_df.to_csv('new_data.csv')

In [29]:
pics_emotions_df = df.loc[:, '#1':'# 12']
pics_emotions_df.head()

,#1,# 2,# 3,# 4,# 5,# 6,# 7,# 8,# 9,# 10,# 11,# 12
0,gemstone,waterfall,clouds,bacon,water,wave,rain,mountains and rivers,volcano,tide,space,explosion
1,planet,galaxy,sea,colored water,rocks,feather,rain,land,planet,ursulla,sand and sea,?
2,galaxy,weird,lock screen photo,ameba,glass,feather,dry paint,earth,australia,3d,sand and sea,?
3,a distant photograph of israel,waves in the sea,waves in the sea,cell development,sea,sea,raindrops on the window,earth,?,?,desert,ink that spreads in water
4,waves cell supernova,cellular structures,wavy,cells (current lesson),beach and sea,dawn,flowing,earth from above,a city,something dark in his nature,dunes,ghost pokémon


In [30]:
for col in pics_emotions_df.columns:
    for indx in pics_emotions_df.index:
        words = pics_emotions_df.loc[indx, col].replace('-', ' ').replace('/', ' ').replace('.', ' ').replace('  ', ' ').split(' ')
        words = list(words)
        for word in words:
            for key in feels_dct:
                for value in feels_dct[key]:
                    if word in value:
                        pics_emotions_df.loc[indx, col] = 1
            for key in cat_dct:
                for value in cat_dct[key]:
                    if word in value:
                        pics_emotions_df.loc[indx, col] = 0
            for key in other_dct:
                for value in other_dct[key]:
                    if word in value:
                        pics_emotions_df.loc[indx, col] = 0
                
pics_emotions_df.head(10)

,#1,# 2,# 3,# 4,# 5,# 6,# 7,# 8,# 9,# 10,# 11,# 12
0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,0,0,0,0,1,1,1,1
6,0,1,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,1,1,0,1,0,0,0,0,1,1,0,1
9,1,1,1,0,0,0,0,0,0,0,1,1


In [31]:
em_df = df.copy()
em_df['#1'] = pics_emotions_df['#1']
em_df['# 2'] = pics_emotions_df['# 2']
em_df['# 3'] = pics_emotions_df['# 3']
em_df['# 4'] = pics_emotions_df['# 4']
em_df['# 5'] = pics_emotions_df['# 5']
em_df['# 6'] = pics_emotions_df['# 6']
em_df['# 7'] = pics_emotions_df['# 7']
em_df['# 8'] = pics_emotions_df['# 8']
em_df['# 9'] = pics_emotions_df['# 9']
em_df['# 10'] = pics_emotions_df['# 10']
em_df['# 11'] = pics_emotions_df['# 11']
em_df['# 12'] = pics_emotions_df['# 12']

em_df.head()

,Age,Sex,Marital Status,Education,#1,# 2,# 3,# 4,# 5,# 6,...,# 8,# 9,# 10,# 11,# 12,O,C,E,A,N
0,2,0,0,2,0,0,0,0,0,0,...,0,0,0,0,1,2,5,1,4,3
1,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,3,4,3,4,3
2,0,1,3,2,0,1,0,0,0,0,...,0,0,0,0,0,3,4,2,4,3
3,0,0,3,2,0,0,0,0,0,0,...,0,0,0,0,0,4,4,3,5,2
4,0,1,3,2,0,0,0,0,0,0,...,0,0,0,0,0,5,3,4,5,2


In [38]:
em_df['user_emo'] = em_df.loc[:, '#1':'# 12'].sum(axis = 1)

em_df.head(10)

,Age,Sex,Marital Status,Education,#1,# 2,# 3,# 4,# 5,# 6,...,# 9,# 10,# 11,# 12,O,C,E,A,N,user_emo
0,2,0,0,2,0,0,0,0,0,0,...,0,0,0,1,2,5,1,4,3,1.0
1,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,3,4,3,4,3,0.0
2,0,1,3,2,0,1,0,0,0,0,...,0,0,0,0,3,4,2,4,3,1.0
3,0,0,3,2,0,0,0,0,0,0,...,0,0,0,0,4,4,3,5,2,0.0
4,0,1,3,2,0,0,0,0,0,0,...,0,0,0,0,5,3,4,5,2,0.0
5,0,0,3,4,1,1,1,1,0,0,...,1,1,1,1,3,4,1,3,2,8.0
6,0,0,3,0,0,1,0,0,0,0,...,0,0,0,0,4,3,5,5,3,1.0
7,0,0,3,2,0,0,0,0,0,0,...,0,0,0,0,3,5,2,4,4,0.0
8,0,2,3,0,1,1,0,1,0,0,...,1,1,0,1,4,3,2,4,4,6.0
9,0,1,3,2,1,1,1,0,0,0,...,0,0,1,1,3,3,3,4,3,5.0


In [39]:
em_df.to_csv('emotion_data.csv')

In [40]:
pics_feelings_df = df.loc[:, '#1':'# 12']
pics_feelings_df.head()

,#1,# 2,# 3,# 4,# 5,# 6,# 7,# 8,# 9,# 10,# 11,# 12
0,gemstone,waterfall,clouds,bacon,water,wave,rain,mountains and rivers,volcano,tide,space,explosion
1,planet,galaxy,sea,colored water,rocks,feather,rain,land,planet,ursulla,sand and sea,?
2,galaxy,weird,lock screen photo,ameba,glass,feather,dry paint,earth,australia,3d,sand and sea,?
3,a distant photograph of israel,waves in the sea,waves in the sea,cell development,sea,sea,raindrops on the window,earth,?,?,desert,ink that spreads in water
4,waves cell supernova,cellular structures,wavy,cells (current lesson),beach and sea,dawn,flowing,earth from above,a city,something dark in his nature,dunes,ghost pokémon


In [41]:
for col in pics_feelings_df.columns:
    for indx in pics_feelings_df.index:
        words = pics_feelings_df.loc[indx, col].replace('-', ' ').replace('/', ' ').replace('.', ' ').replace('  ', ' ').split(' ')
        words = list(words)
        for word in words:
            for key in feels_dct:
                for value in feels_dct[key]:
                    if word in value:
                        pics_feelings_df.loc[indx, col] = key
            for key in cat_dct:
                for value in cat_dct[key]:
                    if word in value:
                        pics_feelings_df.loc[indx, col] = -1
            for key in other_dct:
                for value in other_dct[key]:
                    if word in value:
                        pics_feelings_df.loc[indx, col] = -1
                
pics_feelings_df.head(10)

,#1,# 2,# 3,# 4,# 5,# 6,# 7,# 8,# 9,# 10,# 11,# 12
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,1,0,0,1,-1,-1,-1,-1,1,0,1,0
6,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,1,0,-1,0,-1,-1,-1,-1,0,0,-1,0
9,0,0,0,-1,-1,-1,-1,-1,-1,-1,0,0


In [48]:
feel_df = df.copy()
feel_df['#1'] = pics_feelings_df['#1']
feel_df['# 2'] = pics_feelings_df['# 2']
feel_df['# 3'] = pics_feelings_df['# 3']
feel_df['# 4'] = pics_feelings_df['# 4']
feel_df['# 5'] = pics_feelings_df['# 5']
feel_df['# 6'] = pics_feelings_df['# 6']
feel_df['# 7'] = pics_feelings_df['# 7']
feel_df['# 8'] = pics_feelings_df['# 8']
feel_df['# 9'] = pics_feelings_df['# 9']
feel_df['# 10'] = pics_feelings_df['# 10']
feel_df['# 11'] = pics_feelings_df['# 11']
feel_df['# 12'] = pics_feelings_df['# 12']

negative_feelings = []
positive_feelings = []
for index in feel_df.index:
    neg = 0;
    pos = 0;
    for col in ['#1', '# 2', '# 3', '# 4', '# 5', '# 6', '# 7', '# 8', '# 9', '# 10', '# 11', '# 12']:
        if feel_df.loc[index, col] == '0':
            neg = neg + 1
        elif feel_df.loc[index, col] == '1':
            pos = pos + 1
    negative_feelings.append(neg)
    positive_feelings.append(pos)
    
feel_df['negative'] = negative_feelings
feel_df['positive'] = positive_feelings

feel_df.head()

,Age,Sex,Marital Status,Education,#1,# 2,# 3,# 4,# 5,# 6,...,# 10,# 11,# 12,O,C,E,A,N,negative,positive
0,2,0,0,2,-1,-1,-1,-1,-1,-1,...,-1,-1,0,2,5,1,4,3,1,0
1,0,0,3,0,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,3,4,3,4,3,0,0
2,0,1,3,2,-1,0,-1,-1,-1,-1,...,-1,-1,-1,3,4,2,4,3,1,0
3,0,0,3,2,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,4,4,3,5,2,0,0
4,0,1,3,2,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,5,3,4,5,2,0,0


In [49]:
feel_df.to_csv('feelings_data.csv')